## Python Power: Leveraging ArcGIS Notebooks for Efficient Data ETL

Jin Yao, 4/14/2024

Demo Notebook for the 2024 MidAmerica GIS Symposium

### Table of content

- [1. Import ACS data using Census API](#1.-Import-ACS-data-using-Census-API)
- [2. Clean up data](#2.-Clean-up-data)
    - [2.1 Data type checking and conversion](#2.1-Data-type-checking-and-conversion)
    - [2.2 Handle NULL and negative values](#2.2-Handle-NULL-and-negative-values)
- [3. Export data to Microsoft SQL](#3.-Export-data-to-Microsoft-SQL)

In [1]:
# import packages

import requests
import pandas as pd
from sqlalchemy import create_engine
import time

In [2]:
# Set display options for floating-point display
pd.set_option('display.float_format', '{:.2f}'.format)

### 1. Import ACS data using Census API

Census API for 2022 ACS 5-year estimates: http://api.census.gov/data/2022/acs/acs5.html

In [3]:
# The full API URL
full_url = "https://api.census.gov/data/2022/acs/acs5?get=GEO_ID,B25010_001E,B25010_002E,B25010_003E&for=tract:*&in=state:20;county:091"

# Making a GET request to the full URL
response = requests.get(full_url)

# Print response status. 200 means success
print('API request status: ', response.status_code)

# Print first 250 characters of the response
print('API request return, first 250 characters:')
print(response.text[0:250])

# If request is successfull, save request response
if response.status_code == 200:
    data = response.json()  # Converts the JSON response into a Python dictionary/list
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")

API request status:  200
API request return, first 250 characters:
[["GEO_ID","B25010_001E","B25010_002E","B25010_003E","state","county","tract"],
["1400000US20091050000","2.43","2.50","2.16","20","091","050000"],
["1400000US20091050100","2.11","2.10","2.13","20","091","050100"],
["1400000US20091050200","2.06","2.09


In [4]:
# the first sublist contains column names
col_names = data[0]
col_names

['GEO_ID',
 'B25010_001E',
 'B25010_002E',
 'B25010_003E',
 'state',
 'county',
 'tract']

In [5]:
# Rename the original column names with meaningful names
new_names = ['GEO_ID', 'AHS_all', 'AHS_owner', 'AHS_renter', 'state',  'county', 'tract']
new_names

['GEO_ID', 'AHS_all', 'AHS_owner', 'AHS_renter', 'state', 'county', 'tract']

In [6]:
# construct a dataframe from json
df = pd.DataFrame(columns = new_names, data = data[1:])

print(df.shape)

df.head()

(154, 7)


,GEO_ID,AHS_all,AHS_owner,AHS_renter,state,county,tract
0,1400000US20091050000,2.43,2.50,2.16,20,091,050000
1,1400000US20091050100,2.11,2.10,2.13,20,091,050100
2,1400000US20091050200,2.06,2.09,1.97,20,091,050200
3,1400000US20091050301,1.62,2.00,1.52,20,091,050301
4,1400000US20091050302,1.66,1.90,1.42,20,091,050302


Back to [Table of content](#Table-of-content)

### 2. Clean up data

#### 2.1 Data type checking and conversion

In [7]:
# look at column data type
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   GEO_ID      154 non-null    object
 1   AHS_all     154 non-null    object
 2   AHS_owner   154 non-null    object
 3   AHS_renter  154 non-null    object
 4   state       154 non-null    object
 5   county      154 non-null    object
 6   tract       154 non-null    object
dtypes: object(7)
memory usage: 8.5+ KB


In [8]:
# modify column 'GEO_ID'
# Trim off the first part of GeoID
df['GeoID'] = df['GEO_ID'].str.split('US').str[1]

df.head()

,GEO_ID,AHS_all,AHS_owner,AHS_renter,state,county,tract,GeoID
0,1400000US20091050000,2.43,2.50,2.16,20,091,050000,20091050000
1,1400000US20091050100,2.11,2.10,2.13,20,091,050100,20091050100
2,1400000US20091050200,2.06,2.09,1.97,20,091,050200,20091050200
3,1400000US20091050301,1.62,2.00,1.52,20,091,050301,20091050301
4,1400000US20091050302,1.66,1.90,1.42,20,091,050302,20091050302


In [9]:
# convert the variables that should be float from str to float

vars_float = ['AHS_all', 'AHS_owner', 'AHS_renter']

# covert
df[vars_float] = df[vars_float].astype(float)

df.head()

,GEO_ID,AHS_all,AHS_owner,AHS_renter,state,county,tract,GeoID
0,1400000US20091050000,2.43,2.50,2.16,20,091,050000,20091050000
1,1400000US20091050100,2.11,2.10,2.13,20,091,050100,20091050100
2,1400000US20091050200,2.06,2.09,1.97,20,091,050200,20091050200
3,1400000US20091050301,1.62,2.00,1.52,20,091,050301,20091050301
4,1400000US20091050302,1.66,1.90,1.42,20,091,050302,20091050302


In [10]:
# check column data type again
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   GEO_ID      154 non-null    object 
 1   AHS_all     154 non-null    float64
 2   AHS_owner   154 non-null    float64
 3   AHS_renter  154 non-null    float64
 4   state       154 non-null    object 
 5   county      154 non-null    object 
 6   tract       154 non-null    object 
 7   GeoID       154 non-null    object 
dtypes: float64(3), object(5)
memory usage: 9.8+ KB


#### 2.2 Handle NULL and negative values

In [11]:
# which columns have NULL values?  How many NULL values are in each column
df.isnull().sum().sort_values(ascending = False)

GEO_ID        0
AHS_all       0
AHS_owner     0
AHS_renter    0
state         0
county        0
tract         0
GeoID         0
dtype: int64

In [12]:
# which columns have negative values?  How many negative values are in each column?
df[vars_float].lt(0).sum().sort_values(ascending = False)

AHS_renter    9
AHS_owner     5
AHS_all       4
dtype: int64

In [13]:
# calculate discriptive stats for the numerical variables
df.describe()

,AHS_all,AHS_owner,AHS_renter
count,154.00,154.00,154.00
mean,-17316014.82,-21645019.02,-38961036.77
std,106384484.02,118544334.47,156894573.17
min,-666666666.00,-666666666.00,-666666666.00
25%,2.18,2.33,1.67
50%,2.48,2.67,2.05
75%,2.85,2.98,2.59
max,4.16,3.71,5.08


In [14]:
# look at the rows with negative numbers
cond = (df['AHS_all'] < 0) | (df['AHS_owner'] < 0) | (df['AHS_renter'] < 0)

df.loc[cond, :]

,GEO_ID,AHS_all,AHS_owner,AHS_renter,state,county,tract,GeoID
9,1400000US20091050800,3.03,3.05,-666666666.00,20,091,050800,20091050800
48,1400000US20091052411,2.96,2.96,-666666666.00,20,091,052411,20091052411
69,1400000US20091052612,2.90,2.90,-666666666.00,20,091,052612,20091052612
120,1400000US20091053428,3.48,3.48,-666666666.00,20,091,053428,20091053428
123,1400000US20091053431,2.77,2.85,-666666666.00,20,091,053431,20091053431
149,1400000US20091980001,-666666666.00,-666666666.00,-666666666.00,20,091,980001,20091980001
150,1400000US20091980003,-666666666.00,-666666666.00,-666666666.00,20,091,980003,20091980003
151,1400000US20091980004,4.16,-666666666.00,4.16,20,091,980004,20091980004
152,1400000US20091980005,-666666666.00,-666666666.00,-666666666.00,20,091,980005,20091980005
153,1400000US20091980100,-666666666.00,-666666666.00,-666666666.00,20,091,980100,20091980100


In [15]:
# assign the negative values to NULL
df2 = df.copy()
df2.mask(df2[vars_float] < 0, inplace = True)
df2.head()

,GEO_ID,AHS_all,AHS_owner,AHS_renter,state,county,tract,GeoID
0,1400000US20091050000,2.43,2.50,2.16,20,091,050000,20091050000
1,1400000US20091050100,2.11,2.10,2.13,20,091,050100,20091050100
2,1400000US20091050200,2.06,2.09,1.97,20,091,050200,20091050200
3,1400000US20091050301,1.62,2.00,1.52,20,091,050301,20091050301
4,1400000US20091050302,1.66,1.90,1.42,20,091,050302,20091050302


In [16]:
# look at the rows with negative numbers
df2.loc[cond, :]

,GEO_ID,AHS_all,AHS_owner,AHS_renter,state,county,tract,GeoID
9,1400000US20091050800,3.03,3.05,NaN,20,091,050800,20091050800
48,1400000US20091052411,2.96,2.96,NaN,20,091,052411,20091052411
69,1400000US20091052612,2.90,2.90,NaN,20,091,052612,20091052612
120,1400000US20091053428,3.48,3.48,NaN,20,091,053428,20091053428
123,1400000US20091053431,2.77,2.85,NaN,20,091,053431,20091053431
149,1400000US20091980001,NaN,NaN,NaN,20,091,980001,20091980001
150,1400000US20091980003,NaN,NaN,NaN,20,091,980003,20091980003
151,1400000US20091980004,4.16,NaN,4.16,20,091,980004,20091980004
152,1400000US20091980005,NaN,NaN,NaN,20,091,980005,20091980005
153,1400000US20091980100,NaN,NaN,NaN,20,091,980100,20091980100


In [17]:
# look at descriptive stats again
df2.describe()

,AHS_all,AHS_owner,AHS_renter
count,150.00,149.00,145.00
mean,2.54,2.69,2.29
std,0.46,0.42,0.80
min,1.61,1.69,1.14
25%,2.21,2.35,1.72
50%,2.49,2.68,2.11
75%,2.87,2.99,2.62
max,4.16,3.71,5.08


In [18]:
# select the columns I want to keep
selcol = ['GeoID', 'AHS_all', 'AHS_owner', 'AHS_renter']

df3 = df2[selcol]

print(df3.shape)

df3.head()

(154, 4)


,GeoID,AHS_all,AHS_owner,AHS_renter
0,20091050000,2.43,2.50,2.16
1,20091050100,2.11,2.10,2.13
2,20091050200,2.06,2.09,1.97
3,20091050301,1.62,2.00,1.52
4,20091050302,1.66,1.90,1.42


Back to [Table of content](#Table-of-content)

### 3. Export data to Microsoft SQL

In [19]:
# define output sql server, database, and table name

servername = '________'  # fill in your sql server name
dbname = '_________'     # fill in your database name

# output sql table name
sqltb_out = 'jy_demo_AvgHouseholdSize'

In [20]:
# create engine to connect to our database
# use Windows Authentication with sqlalchemy and mssql

engine = create_engine('mssql+pyodbc://@' + 
                       servername + '/' + 
                       dbname + 
                       '?trusted_connection=yes&driver=ODBC+Driver+13+for+SQL+Server')

In [21]:
# write to a sql table
print('Write df to sql ...')

start = time.time()
df3.to_sql(sqltb_out, 
           engine, schema = 'dbo', 
           if_exists = 'replace', index = False)

end = time.time()
print(end - start, ' sec')

Write df to sql ...
0.21306061744689941  sec


Back to [Table of content](#Table-of-content)